In [1]:
import torch
from torch import nn
from torch.autograd import Variable

from data_loader import DataLoader
from model import FillThoughts
from config import *
from datetime import datetime, timedelta

from tensorboardX import SummaryWriter
import os

In [2]:
language = "english"
d = DataLoader("../dataset/"+language+"/all.tokenized.txt")

Loading text file at ../dataset/english/all.tokenized.txt
Making dictionary for these words
Using cached dictionary at ../dataset/english/all.tokenized.txt.pkl
Making reverse dictionary


In [3]:
mod = FillThoughts()
if USE_CUDA:
    print("USING CUDA")
    mod.cuda(CUDA_DEVICE)

USING CUDA


/jet/var/python/lib/python3.6/site-packages/torch/backends/cudnn/__init__.py:89: UserWarning: PyTorch was compiled without cuDNN support. To use cuDNN, rebuild PyTorch making sure the library is visible to the build system.
  "PyTorch was compiled without cuDNN support. To use cuDNN, rebuild "


In [4]:
lr = 3e-4
optimizer = torch.optim.Adam(params=mod.parameters(), lr=lr)

In [5]:
writer = SummaryWriter()

In [6]:
loss_trail = []
last_best_loss = None
start_i = 0

def debug(i, loss, sent, pred):
    global loss_trail
    global last_best_loss
    global current_time

    this_loss = loss.item()
    loss_trail.append(this_loss)
    loss_trail = loss_trail[-20:]
    
    try:
        trail_loss = sum(loss_trail)/len(loss_trail)
        if last_best_loss is None or last_best_loss > trail_loss:
            print("Loss improved from {} to {}".format(last_best_loss, trail_loss))
            
            print("saving model at {}".format(save_loc))
            torch.save(mod.state_dict(), save_loc)
            torch.save((i, trail_loss), save_loc+".meta")
            
            last_best_loss = trail_loss
    except Exception as e:
       print("Couldn't save model because {}".format(e))
    
def get_natural_sentence(res):
    sent = [x for x in res if x != 0]
    sent = d.convert_indices_to_sentences(sent)

    from sacremoses import MosesDetokenizer
    detokenizer = MosesDetokenizer()

    return detokenizer.detokenize(sent.split(" "), return_str=True)

In [7]:
reload_last_checkpoint = False

save_loc = "./saved_models/skip-best-{}-{}".format(lr, VOCAB_SIZE)
    
if reload_last_checkpoint and os.path.exists(save_loc):
    print("Loading last checkpoint: "+save_loc)
    mod.load_state_dict(torch.load(save_loc))
    
    start_i, last_best_loss = torch.load(save_loc+".meta")
    print("At iter "+str(start_i)+" | Loss: " + str(last_best_loss))

In [8]:
print("Starting training...")

# a million iterations
for i in range(start_i, 1000000):
    sentences, lengths = d.fetch_batch(32 * 8)

    loss, sent, pred  = mod(sentences, lengths)
    writer.add_scalar('loss', loss, i)
    

    if i % 30 == 0:
        print(i)
        nat_sent = get_natural_sentence(sent)
        nat_pred = get_natural_sentence(pred)
        
        writer.add_text('Sentence', nat_sent + ' |> ' + nat_pred, i)
        debug(i, loss, sent, pred)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

/jet/var/python/lib/python3.6/site-packages/torch/nn/functional.py:995: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/jet/var/python/lib/python3.6/site-packages/torch/backends/cudnn/__init__.py:89: UserWarning: PyTorch was compiled without cuDNN support. To use cuDNN, rebuild PyTorch making sure the library is visible to the build system.
  "PyTorch was compiled without cuDNN support. To use cuDNN, rebuild "


Starting training...
0
Loss improved from None to 9.890910148620605
saving model at ./saved_models/skip-best-0.0003-20000


KeyboardInterrupt: 

In [ ]:
print(i)
print("saving model at {}".format(save_loc))
trail_loss = sum(loss_trail)/len(loss_trail)
torch.save(mod.state_dict(), save_loc)
torch.save((i, trail_loss), save_loc+".meta")